[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [64]:
# 라이브러리 로드
import pandas as pd
import numpy as np

## 수집할 URL 정하기

In [65]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "하이브"

item_code = "326030"
item_name = "SK바이오팜"

# 종목 URL 만들기
# url = "https://finance.naver.com/item/sise.nhn?code=005930"
url = "https://finance.naver.com/item/sise_day.nhn?code=005930&page=10"

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [66]:
import requests
requests.__version__

'2.25.1'

In [67]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
print(type(headers))
headers

<class 'dict'>


{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}

In [68]:
response = requests.get(url, headers = headers)
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [69]:
from bs4 import BeautifulSoup as bs
html = bs(response.text, "lxml")

In [70]:
tables = html.select("table")
len(tables)

2

In [71]:
import tqdm
tqdm.__version__

'4.59.0'

## pandas 코드 한 줄로 데이터 수집하기

In [72]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=10


In [73]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.

table = pd.read_html(str(tables))
table

[            날짜       종가     전일비       시가       고가       저가         거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 1   2021.03.22  82000.0   100.0  82000.0  82300.0  81700.0  12670506.0
 2   2021.03.19  81900.0  1000.0  82100.0  82500.0  81800.0  15869700.0
 3   2021.03.18  82900.0   600.0  82800.0  83800.0  82600.0  18585244.0
 4   2021.03.17  82300.0   500.0  82800.0  82900.0  82000.0  11625146.0
 5   2021.03.16  82800.0  1000.0  82200.0  83000.0  82100.0  12293537.0
 6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 9   2021.03.15  81800.0  1000.0  82800.0  82900.0  81800.0  13735798.0
 10  2021.03.12  82800.0   800.0  83100.0  83500.0  82400.0  13530573.0
 11  2021.03.11  82000.0  1100.0  81000.0  82500.0  81000.0  23818297.0
 12  2021.03.10  80900.0   500.0  82400.0  82500.0  80700.0  200

In [74]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.03.22,82000.0,100.0,82000.0,82300.0,81700.0,12670506.0
2,2021.03.19,81900.0,1000.0,82100.0,82500.0,81800.0,15869700.0
3,2021.03.18,82900.0,600.0,82800.0,83800.0,82600.0,18585244.0
4,2021.03.17,82300.0,500.0,82800.0,82900.0,82000.0,11625146.0
5,2021.03.16,82800.0,1000.0,82200.0,83000.0,82100.0,12293537.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.03.15,81800.0,1000.0,82800.0,82900.0,81800.0,13735798.0


In [75]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.03.22,82000.0,100.0,82000.0,82300.0,81700.0,12670506.0
2,2021.03.19,81900.0,1000.0,82100.0,82500.0,81800.0,15869700.0
3,2021.03.18,82900.0,600.0,82800.0,83800.0,82600.0,18585244.0
4,2021.03.17,82300.0,500.0,82800.0,82900.0,82000.0,11625146.0
5,2021.03.16,82800.0,1000.0,82200.0,83000.0,82100.0,12293537.0
9,2021.03.15,81800.0,1000.0,82800.0,82900.0,81800.0,13735798.0
10,2021.03.12,82800.0,800.0,83100.0,83500.0,82400.0,13530573.0
11,2021.03.11,82000.0,1100.0,81000.0,82500.0,81000.0,23818297.0
12,2021.03.10,80900.0,500.0,82400.0,82500.0,80700.0,20038422.0
13,2021.03.09,81400.0,600.0,81400.0,81900.0,80600.0,25420764.0


## 페이지별 데이터 수집 함수 만들기

In [76]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=10


In [77]:
headers

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}

In [78]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
#     print(url)
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}
    response = requests.get(url, headers=headers)
    html = bs(response.text, 'lxml')
    tables = html.select("table")
    table = pd.read_html(str(tables))
    df_page = table[0].dropna()
    return df_page

In [79]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 20
get_day_list(item_code, page_no)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.10.23,161000.0,1500.0,161000.0,162000.0,159500.0,191225.0
2,2020.10.22,162500.0,1000.0,161000.0,163500.0,158000.0,287992.0
3,2020.10.21,161500.0,6500.0,156000.0,162000.0,154500.0,449038.0
4,2020.10.20,155000.0,500.0,153500.0,156000.0,152500.0,232226.0
5,2020.10.19,154500.0,1000.0,151000.0,156000.0,149000.0,270633.0
9,2020.10.16,153500.0,1500.0,153000.0,159000.0,153000.0,486968.0
10,2020.10.15,152000.0,1000.0,153000.0,153000.0,146500.0,402884.0
11,2020.10.14,153000.0,5500.0,166500.0,170000.0,152500.0,1784912.0
12,2020.10.13,147500.0,2500.0,151000.0,151500.0,146000.0,301158.0
13,2020.10.12,150000.0,2500.0,149000.0,150000.0,147500.0,369263.0


## 종목토론실, 뉴스공시 수집 하신 분들은 스크린샷을 스레드로 올려주세요.

In [80]:
### 뉴스공시 수집
def find_day_news(item_code, page_num):
    """
    일자별 종목 뉴스 수집
    """
    url = f"https://finance.naver.com/item/news_news.nhn?code={item_code}&page={page_num}&sm=title_entity_id.basic&clusterId="
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
    response = requests.get(url, headers = headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables))
    news_page = table[0]
    news_page_result = news_page[~news_page.get("제목").str.contains("연관기사")]
    return news_page_result

In [81]:
# 함수가 잘 만들어졌는지 확인하기
page_num = 20
find_day_news(item_code, page_num)

,제목,정보제공,날짜
0,다음달 3.2억주 의무보유 해제…SK바이오팜 5873만주 풀린다,조선비즈,2020.12.31 11:09
2,내달 코스피·코스닥 3.2억株 의무보유 해제…수량 1위 ‘SK바이오팜...,이데일리,2020.12.31 09:47
3,"SK바이오팜 ""중국 등 아시아로 글로벌 파트너십 확대""",한국경제,2020.12.29 14:19
4,"SK바이오팜 5,800만 주 보호예수 내년 1월 1일 종료",서울경제,2020.12.29 11:32
5,"[2020자본투자대상]한투證, SK바이오팜 등 빅3 상장 주관 IPO ...",아시아경제,2020.12.09 14:00
6,"[마켓인사이트]‘3조’ 대어 SK바이오사이언스, 상장 준비 본격화......",한국경제,2020.12.04 11:00
7,"SK바이오사이언스, 코스피 상장 추진…제2의 SK바이오팜 될까",머니투데이,2020.11.27 16:56
8,올해 공모주 수익률 상위권 휩쓴 바이오株… 상승률 1위는 SK바이오팜,조선비즈,2020.11.27 14:00
9,"SK바이오팜 ""세노바메이트 美 승인 1주년…월평균 2260건 처방""",한국경제,2020.11.23 09:46
11,"SK바이오팜 뇌전증 혁신신약 세노바메이트, 美 FDA 승인 1주년 맞...",파이낸셜뉴스,2020.11.23 08:56


In [82]:
### 종목 토론 수집
def get_day_debate(item_code, page_num):
    """
    종목 토론 수집
    """
    url = f"https://finance.naver.com/item/board.nhn?code=005930&page={page_num}"
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
    response = requests.get(url, headers = headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables))
    table_page = table[1]
    df_debate = table_page.iloc[:,:6]
    debate_page_result = df_debate.dropna()
    return debate_page_result

In [83]:
# 함수가 잘 만들어졌는지 확인하기
page_num = 20
get_day_debate(item_code, page_num)

,날짜,제목,글쓴이,조회,공감,비공감
1,2021.07.28 09:10,어째 하는짓거리가 6만원대 찍을거 같습니...,krpc****,71.0,0.0,1.0
2,2021.07.28 09:10,손절 버튼 누르지말고 참아봐라,yang****,74.0,2.0,0.0
3,2021.07.28 09:10,56도 비싸다 [1],ciel****,110.0,2.0,3.0
4,2021.07.28 09:10,벌허,ct27****,46.0,1.0,1.0
5,2021.07.28 09:10,■썩은 반자본주의 좌파사상과 착취로 국...,perp****,41.0,0.0,0.0
7,2021.07.28 09:10,답답하네,seji****,49.0,0.0,0.0
8,2021.07.28 09:09,오늘도 [1],gose****,82.0,0.0,0.0
9,2021.07.28 09:09,삼전 45000부터 분할 매수 시작하시면...,capp****,87.0,1.0,2.0
10,2021.07.28 09:09,안되겠다ㅋㅋㄲ,tmda****,52.0,0.0,0.0
11,2021.07.28 09:09,거지가 되고싶니?????,mado****,59.0,0.0,0.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [84]:
%%time
print(1)
time.sleep(3)
print(2)

1
2
Wall time: 3.01 s


In [85]:
url

'https://finance.naver.com/item/sise_day.nhn?code=005930&page=10'

In [86]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []
for page_no in range(page_no,10):
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    print(page_no)
    
item_list

1
2
3
4
5
6
7
8
9


[            날짜        종가     전일비        시가        고가        저가       거래량
 1   2021.07.28  121000.0  1000.0  121500.0  122500.0  121000.0   54417.0
 2   2021.07.27  122000.0  1500.0  121000.0  122500.0  120500.0  146705.0
 3   2021.07.26  120500.0  1500.0  122500.0  123000.0  120000.0  221231.0
 4   2021.07.23  122000.0  2000.0  124000.0  125000.0  122000.0  235444.0
 5   2021.07.22  124000.0  1000.0  125000.0  125000.0  123000.0  155904.0
 9   2021.07.21  125000.0   500.0  125500.0  127000.0  124000.0  194282.0
 10  2021.07.20  124500.0  1000.0  123500.0  125500.0  123000.0  198884.0
 11  2021.07.19  125500.0     0.0  124000.0  127000.0  123000.0  216944.0
 12  2021.07.16  125500.0  3000.0  127500.0  128500.0  125500.0  327694.0
 13  2021.07.15  128500.0  2000.0  130500.0  131500.0  128000.0  228926.0,
             날짜        종가     전일비        시가        고가        저가        거래량
 1   2021.07.14  130500.0  2000.0  132500.0  132500.0  129500.0   214600.0
 2   2021.07.13  132500.0  5000.0  

In [87]:
import time
from tqdm import trange
# web page 시작번호
start_no = 1
end_no = 9
# 데이터를 저장할 빈 변수 선언
item_list = []
for page_no in trange(start_no,end_no+1):
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
# item_list

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  8.48it/s]


In [88]:
item_code = '326030'
item_name = 'SK바이오팜'

# 페이지 별 과를 받을 리스트를 초기화
item_list = []

# while
page_no = 1

while True:
    # 수집함수
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    # 서버에 부담을 주지 않기 위해 시간차를 두고 가져옴
    random_time = np.random.uniform(0.1, 0.5)
    time.sleep(random_time)
    
    # 페이지 번호를 하나씩 증가 시키고
    page_no = page_no + 1
    
    # 5페이지 단위로 현재 몇페이지인지를 출력
    if page_no % 5 == 0:
        print(page_no)
        
    # 일별시세는 10개씩 행이 있기 때문에, 행이 10개보다 적으면 마지막 페이지로 보고 break
    if temp.shape[0] != 10:
        break

5
10
15
20
25


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [89]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [90]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.07.28,121000.0,1000.0,121500.0,122500.0,121000.0,54417.0
2,2021.07.27,122000.0,1500.0,121000.0,122500.0,120500.0,146705.0
3,2021.07.26,120500.0,1500.0,122500.0,123000.0,120000.0,221231.0
4,2021.07.23,122000.0,2000.0,124000.0,125000.0,122000.0,235444.0
5,2021.07.22,124000.0,1000.0,125000.0,125000.0,123000.0,155904.0


In [91]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
3,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
4,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
5,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
9,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0
10,2020.07.02,127000.0,29000.0,98000.0,127000.0,98000.0,698642.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [92]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name

In [93]:
df.head(2)

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.07.28,121000.0,1000.0,121500.0,122500.0,121000.0,54417.0,326030,SK바이오팜
2,2021.07.27,122000.0,1500.0,121000.0,122500.0,120500.0,146705.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [94]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [95]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df.head(2)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,326030,SK바이오팜,2021.07.28,121000.0,1000.0,121500.0,122500.0,121000.0,54417.0
2,326030,SK바이오팜,2021.07.27,122000.0,1500.0,121000.0,122500.0,120500.0,146705.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [96]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
print(df.shape)
df.drop_duplicates()
df.shape
# 중복 데이터 없음을 확인

(267, 9)


(267, 9)

## 기술통계값 구하기

In [97]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,267.000000,267.000000,267.000000,267.000000,267.000000,2.670000e+02
mean,146775.280899,3059.925094,147204.119850,149719.101124,144250.936330,5.912933e+05
std,29285.444873,4921.199040,29689.883689,30863.964960,28628.432898,1.166652e+06
min,101000.000000,0.000000,98000.000000,103000.000000,98000.000000,5.441700e+04
25%,118750.000000,1000.000000,118250.000000,120750.000000,116000.000000,2.116635e+05
50%,150500.000000,2000.000000,151000.000000,153000.000000,149000.000000,3.155770e+05
75%,171000.000000,3500.000000,171000.000000,173000.000000,168500.000000,5.579945e+05
max,217000.000000,49500.000000,219000.000000,269500.000000,210000.000000,1.242668e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [98]:
# 거래량의 과학적 기수법 읽기
3.616800e+04

36168.0

## 최근 날짜 구해서 파일명 만들기

In [99]:
# 날짜 column의 첫 row값 확인
# date
date = df.iloc[0, 2]
date

'2021.07.28'

In [100]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2021.07.28.csv'

## 파일로 저장하기

In [101]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index = False)

In [102]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2021.07.28,121000.0,1000.0,121500.0,122500.0,121000.0,54417.0
1,326030,SK바이오팜,2021.07.27,122000.0,1500.0,121000.0,122500.0,120500.0,146705.0
2,326030,SK바이오팜,2021.07.26,120500.0,1500.0,122500.0,123000.0,120000.0,221231.0
3,326030,SK바이오팜,2021.07.23,122000.0,2000.0,124000.0,125000.0,122000.0,235444.0
4,326030,SK바이오팜,2021.07.22,124000.0,1000.0,125000.0,125000.0,123000.0,155904.0
...,...,...,...,...,...,...,...,...,...
262,326030,SK바이오팜,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
263,326030,SK바이오팜,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
264,326030,SK바이오팜,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
265,326030,SK바이오팜,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0


## 전체 과정을 하나의 함수로 만들기

In [110]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # 반복문
    # 페이지 별 과를 받을 리스트를 초기화
    item_list = []

    # 페이지 번호 초기화
    page_no = 1

    while True:
        # 수집함수
        temp = get_day_list(item_code, page_no)
        
        # 두번째 페이지부터 확인
        if page_no > 1:
            pre_date = item_list[-1]['날짜'].tolist()[0]
            now_date = temp['날짜'].tolist()[0]
            if pre_date == now_date :
                break
                
        item_list.append(temp)

        # 서버에 부담을 주지 않기 위해 시간차를 두고 가져옴
        random_time = np.random.uniform(0.1, 0.5)
        time.sleep(random_time)

        # 페이지 번호를 하나씩 증가 시키고
        page_no = page_no + 1

        # 5페이지 단위로 현재 몇페이지인지를 출력
        if page_no % 5 == 0:
            print(page_no)

        # 일별시세는 10개씩 행이 있기 때문에, 행이 10개보다 적으면 마지막 페이지로 보고 break
        if temp.shape[0] != 10:
            break

    # 전처리
    df = pd.concat(item_list)
    df['종목코드'] = item_code
    df['종목명'] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols].copy()
    
    # 파일저장
    date = df.iloc[0,2]
    file_name = f"{item_name}_{item_code}_{date}.csv"
    df.to_csv(file_name, index = False)
    return True

In [111]:
# item_code = "352820"
# item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

item_code = "326030"
item_name = "SK바이오팜"
get_item_list(item_code, item_name)

5
10
15
20
25


True

## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [ ]:
# 도움말 보기
# get_item_list?

In [ ]:
# 소스코드 보기
# get_item_list??